This notebook is used to add synthetic data to datasets.\
The current implemented method is adding random ratings to a target user in a target cluster.

We can take target users from a certain cluster based on the result of the previous notebook (clustering.ipynb) and use this code to generate random ratings for few users inside a given cluster. This simulates an opt-out (or any other data manipulation scenario) scenario.

In [1]:
import os

# change dir for custom imports
os.chdir('../')

In [2]:
import pandas as pd

dataset_name = 'ml-latest-small'

ratings = pd.read_csv('datasets/' + dataset_name + '/clean/ratings.csv')
movies = pd.read_csv('datasets/' + dataset_name + '/movies.csv')
clusters = pd.read_csv('output/' + dataset_name + '/clusters.csv')

ratings_detailed = pd.merge(ratings, movies, how='inner', on='movieId').sort_values(by='userId')
ratings_detailed = pd.merge(ratings_detailed, clusters, how='inner', on=['userId', 'movieId', 'rating', 'timestamp'])

total_movies = len(list(set(ratings.movieId.to_list())))
max_rating = max(list(set(ratings.rating.to_list())))
min_rating = min(list(set(ratings.rating.to_list())))
total_clusters = len(set(clusters.cluster.to_list()))

print(
    "total_movies:\t%f" % total_movies,
    "max_rating:\t%f" % max_rating,
    "min_rating:\t%f" % min_rating,
    "clusters\t%f" % total_clusters, sep='\n'
)

total_movies:	9633.000000
max_rating:	5.000000
min_rating:	0.500000
clusters	50.000000


# Reverse profiles
This code will allow us to reverse user profiles in datasets for a target cluster/s. The idea is equivalent to opt-out where a user masks his identity behind false feedback that's not similar to his initial ratings.

In [3]:
from helpers.dataset_helpers import flip_pofile_ratings

target_cluster = 20
# total number of items to add per each genre (per user)
N = 150
# total number of users to consider for data addition (per cluster)
users_thresh = 50

new_ratings = flip_pofile_ratings(ratings_detailed, movies, target_cluster, N, users_thresh)
new_ratings.to_csv('datasets/' + dataset_name + '/modified/ratings_genre_experiment.csv', index=False)

# Random ratings
This code will us to add random ratings for users in datasets.

In [7]:
from helpers.dataset_helpers import add_random_ratings

# total number of random ratings to add per target user 
target_cluster = 20
N = 2500
# total number of users to consider for data addition (per cluster)
users_thresh = 50

new_ratings = add_random_ratings(ratings, clusters, target_cluster, N, users_thresh)
new_ratings.to_csv('datasets/' + dataset_name + '/modified/ratings_random_experiment.csv', index=False)